In this lesson we will be exploring linked lists.  We will use strings as an example, but the normal C implementation of strings is pretty good.

Always remember that arrays may be a better alternative to linked lists.

In particular, implementing strings as liked lists is very inefficient (9-16 bytes per character), lots of operations to access the characters (especially near the back of the string), etc.

But, it is a simple example that will show us a lot about dynamic memory allocation with pointers.

A linked list is a simple dynamic memory structure that uses structures and pointers.  This is the general set-up.

In [1]:
%%file linked_list.h
struct linked_list_struct
{
    /* data */
    struct linked_list_struct *ptr;  /* points to the next item in the list */
};



Overwriting linked_list.h


For our example we will work with linked lists of characters.  I.e. the data, in the example above, will be a single character.

Here's our structure:

In [2]:
%%file strlst.h
#ifndef STRLST_H
#define STRLST_H
struct strlst_struct
{
    char character;
    struct strlst_struct *next;
};

struct strlst_struct *new_item( char character );
/* Return a pointer to a newly allocated item whose character attribute
   is set to character and whose next pointer is NULL.
*/

char free_item( struct strlst_struct *ptr );
/* Free the memory pointed to by the ptr and return the character 
   attribute in the structure that ptr points to.
*/

void push( struct strlst_struct **list_ptr, 
           struct strlst_struct *item_ptr );
/* Add the item pointed to by item_ptr to the beginning of the list 
   pointed to by list_ptr.
*/

struct strlst_struct *pop( struct strlst_struct **list_ptr );
/* Remove the first item from the list pointed to by list_ptr and return
   a pointer to it.  Set its next pointer to NULL.
*/
#endif

Overwriting strlst.h


In [3]:
%%file new_item.c
#include <stdlib.h>
#include "strlst.h"

struct strlst_struct *new_item( char character )
/* Return a pointer to a newly allocated item whose character attribute
   is set to character and whose next pointer is NULL.
*/
{
    struct strlst_struct *item_ptr;
    
    item_ptr = malloc( sizeof(struct strlst_struct) );
    item_ptr->character = character;
    (*item_ptr).next = NULL;

    return item_ptr;
}

Overwriting new_item.c


In [4]:
%%file free_item.c
#include <stdlib.h>
#include "strlst.h"

char free_item( struct strlst_struct *ptr )
/* Free the memory pointed to by the ptr and return the character 
   attribute in the structure that ptr points to.
*/
{
    char character;
    
    character = ptr->character;
    free( ptr );
    
    return character;
}

Overwriting free_item.c


In [5]:
%%file push.c
#include <stdlib.h>
#include "strlst.h"

void push( struct strlst_struct **list_ptr, 
           struct strlst_struct *item_ptr )
/* Add the item pointed to by item_ptr to the beginning of the list 
   pointed to by list_ptr.
*/
{
    item_ptr->next = *list_ptr;
    *list_ptr = item_ptr;
}

Overwriting push.c


In [6]:
%%file pop.c
#include <stdlib.h>
#include "strlst.h"

struct strlst_struct *pop( struct strlst_struct **list_ptr )
/* Remove the first item from the list pointed to by list_ptr and return
   a pointer to it.  Set its next pointer to NULL.
*/
{
    struct strlst_struct *item;
    
    item = *list_ptr;
    *list_ptr = item->next;
    item->next = NULL;
    
    return item;
}

Overwriting pop.c


In [7]:
%%file main.c
#include "strlst.h"
#include <stdlib.h>
#include <stdio.h>

int main()
{
    char letter;
    char *string = "dlrow olleh\n";
    struct strlst_struct *item_ptr, *list_ptr;
    
    list_ptr = NULL;
    
    for (;*string;string++)
    {
        item_ptr = new_item( *string );
        push( &list_ptr, item_ptr );
    }
    
    while (list_ptr)
    {
        item_ptr = pop( &list_ptr );
        letter = free_item( item_ptr );
        printf( "%c", letter );
    }
    printf( "\n" );
    
    return 0;
}

Overwriting main.c


In [8]:
print("\t");

In [9]:
%%file makefile
list: main.o new_item.o free_item.o push.o pop.o
	gcc -Wall -ansi -pedantic main.o new_item.o free_item.o push.o pop.o -o list
    
main.o: main.c strlst.h
	gcc -Wall -ansi -pedantic -c main.c -o main.o
    
new_item.o: new_item.c strlst.h
	gcc -Wall -ansi -pedantic -c new_item.c -o new_item.o

free_item.o: free_item.c strlst.h
	gcc -Wall -ansi -pedantic -c free_item.c -o free_item.o

push.o: push.c strlst.h
	gcc -Wall -ansi -pedantic -c push.c -o push.o

pop.o: pop.c strlst.h
	gcc -Wall -ansi -pedantic -c pop.c -o pop.o

    

Overwriting makefile


In [10]:
%%bash
make

gcc -Wall -ansi -pedantic -c main.c -o main.o
gcc -Wall -ansi -pedantic -c new_item.c -o new_item.o
gcc -Wall -ansi -pedantic -c free_item.c -o free_item.o
gcc -Wall -ansi -pedantic -c push.c -o push.o
gcc -Wall -ansi -pedantic -c pop.c -o pop.o
gcc -Wall -ansi -pedantic main.o new_item.o free_item.o push.o pop.o -o list


In [11]:
%%bash
./list


hello world


In [7]:
%%file morestrlst.h


#include "strlst.h"
#include <string.h>

struct strlst_struct *strlst_from_str( char *string );
/* create a strlst from a given string */

void free_strlst( struct strlst_struct *strlst );
/* free all the elements of a strlst */

struct strlst_struct *get_item( struct strlst_struct *strlst, int i );
/* retrieve a given item number from the strlst (start counting at 0) */

struct strlst_struct *copy_strlst( struct strlst_struct *strlst );
/* make a copy of a strlst */

void reverse_strlst( struct strlst_struct **strlst );
/* reverse the elements of a strlst */

void append_strlst( struct strlst_struct **destlst, struct strlst_struct *srclst );
/* append srclst to the end of destlst, do not malloc or free anything */

struct strlst_struct *find_strlst( struct strlst_struct *searchlst, struct strlst_struct *target );
/* find the items of target in searchlst */

Overwriting morestrlst.h


In [56]:
%%file morestrlst.c

#include "strlst.h"
#include <string.h>


struct strlst_struct *strlst_from_str( char *string )
/* create a strlst from a given string */
{
    int i;
    struct strlst_struct *item_ptr, *list_ptr;
    
    /* this code contains one error - can we find it? */
    list_ptr = NULL;
    for (i=strlen(string)-1;i>=0;i--)
    {
        item_ptr = new_item(string[i]);
        push( &list_ptr, item_ptr );
    }
    return list_ptr;
}

void free_strlst( struct strlst_struct *strlst )
/* free all the elements of a strlst */
{
    struct strlst_struct *item_ptr;
    
    while (strlst)
    {
        item_ptr = pop( &strlst );
        free_item( item_ptr );
    }
}

struct strlst_struct *get_item( struct strlst_struct *strlst, int i )
/* retrieve a given item number from the strlst (start counting at 0) */
{
    while( strlst )
    {
        if (i==0)
            return strlst;
        strlst = strlst->next;
        i--;
    }
    return NULL;
    
}


struct strlst_struct *copy_strlst( struct strlst_struct *strlst )
/* make a copy of a strlst */
{
    struct strlst_struct *newlist, **next, **from;
    
    newlist = NULL;
    
    from = &strlst;
    next = &newlist;
    

    while (*from)
    {
        *next = new_item( (*from)->character );
        next = &((*next)->next);
        from = &((*from)->next);
    }
    
    return newlist;
    
    
}


void reverse_strlst( struct strlst_struct **strlst );
/* reverse the elements of a strlst */

void append_strlst( struct strlst_struct **destlst, struct strlst_struct *srclst );
/* append srclst to the end of destlst, do not malloc or free anything */

struct strlst_struct *find_strlst( struct strlst_struct *searchlst, struct strlst_struct *target );
/* find the items of target in searchlst */

Overwriting morestrlst.c


In [57]:
%%bash
gcc -Wall -ansi -pedantic -c morestrlst.c -o morestrlst.o

In [58]:
%%file main.c
#include "strlst.h"
#include "morestrlst.h"

#include <stdio.h>

int main()
{
    struct strlst_struct *list, *list_copy, *item;

    
    list = strlst_from_str( "Hello, world\n" );
    item = list;
    while (item)
    {
        printf( "%c", item->character );
        item = item->next;
    }
    
    list_copy = copy_strlst( list );
    free_strlst( list );
    
    item = list_copy;
    while (item)
    {
        printf( "%c", item->character );
        item = item->next;
    }
    
 
    free_strlst( list_copy );
    return 0;
}

Overwriting main.c


In [59]:
%%bash
gcc -ansi -Wall -pedantic -c main.c -o main.o

In [60]:
%%bash
gcc -Wall -pedantic main.o morestrlst.o new_item.o push.o pop.o free_item.o -o list

In [61]:
%%bash
./list

Hello, world
Hello, world
